In [8]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer


In [9]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]




In [10]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, input_shape=(len(trainX[0]),), activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

hist = model.fit(np.array(trainX), np.array(trainY), epochs=300, batch_size=5, verbose=1)

#model.save('chatbot_model.h5', hist)
print('Done')

Epoch 1/300
12/12 [==============================] - 0s 1ms/step - loss: 2.7432 - accuracy: 0.0175
Epoch 2/300
12/12 [==============================] - 0s 870us/step - loss: 2.6875 - accuracy: 0.1228
Epoch 3/300
12/12 [==============================] - 0s 929us/step - loss: 2.6172 - accuracy: 0.2632
Epoch 4/300
12/12 [==============================] - 0s 1ms/step - loss: 2.5600 - accuracy: 0.2982
Epoch 5/300
12/12 [==============================] - 0s 2ms/step - loss: 2.5107 - accuracy: 0.2982
Epoch 6/300
12/12 [==============================] - 0s 2ms/step - loss: 2.4372 - accuracy: 0.3158
Epoch 7/300
12/12 [==============================] - 0s 782us/step - loss: 2.3651 - accuracy: 0.5088
Epoch 8/300
12/12 [==============================] - 0s 786us/step - loss: 2.2823 - accuracy: 0.4737
Epoch 9/300
12/12 [==============================] - 0s 1ms/step - loss: 2.1710 - accuracy: 0.5263
Epoch 10/300
12/12 [==============================] - 0s 2ms/step - loss: 2.0618 - accuracy: 0.5965
E

## Evaluate Model

In [11]:
test = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    test.append(bag + outputRow)

random.shuffle(test)
training = np.array(test)

testX = training[:, :len(words)]
testY = training[:, len(words):]


In [12]:


# Evaluate the model on the test data
loss, accuracy = model.evaluate(testX, testY, verbose=0)
print(f'Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}')

# Predict on test data
predictions = model.predict(testX)
# Calculate precision
true_positives = 0
false_positives = 0

for i in range(len(testY)):
    true_label = np.argmax(testY[i])
    predicted_label = np.argmax(predictions[i])

    if true_label == predicted_label:
        true_positives += 1
    else:
        false_positives += 1

precision = true_positives / (true_positives + false_positives)
print(f'Precision: {precision:.4f}')

Test Loss: 0.1443, Test Accuracy: 0.9298
2/2 [==============================] - 0s 1ms/step
Precision: 0.9298
